In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Responsible AI com a Vertex AI Gemini API: Classificações e limites de segurança

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/responsible-ai/gemini_safety_ratings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Execute no Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/responsible-ai/gemini_safety_ratings.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      Veja no GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/responsible-ai/gemini_safety_ratings.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Abra no Workbench da Vertex AI
    </a>
  </td>                                                                                               
</table>

## Visão geral

Grandes modelos de linguagem (LLMs) podem traduzir idiomas, resumir textos, gerar escrita criativa, gerar código, potencializar chatbots e assistentes virtuais e complementar mecanismos de pesquisa e sistemas de recomendação. A incrível versatilidade dos LLMs também é o que torna difícil prever exatamente que tipos de resultados não intencionais ou imprevistos eles podem produzir.

Considerando esses riscos e complexidades, a API Vertex AI Gemini foi projetada tendo em mente os [Princípios de IA do Google](https://ai.google/responsibility/principles/). No entanto, é importante que os desenvolvedores entendam e testem seus modelos para implantá-los com segurança e responsabilidade. Para ajudar os desenvolvedores, o Vertex AI Studio possui filtragem de conteúdo integrada, classificações de segurança e a capacidade de definir limites de filtro de segurança adequados para seus casos de uso e negócios.

Para mais informações, consulte a documentação do Google Cloud Generative AI sobre [Responsible AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/responsible-ai).

### Objetivos

Neste tutorial, você aprenderá como inspecionar as classificações de segurança retornadas da API Vertex AI Gemini e como definir um limite de segurança para filtrar respostas da API Vertex AI Gemini.

As etapas executadas incluem:

- Chame a API Vertex AI Gemini e inspecione as classificações de segurança das respostas
- Defina um limite para filtrar as classificações de segurança de acordo com suas necessidades

### Custos

Este tutorial usa os seguintes componentes de Google Cloud que podem gerar custos em sua fatura:

- Vertex AI

Saiba mais sobre [preços da Vertex AI](https://cloud.google.com/vertex-ai/pricing) e use a [calculadora de preços](https://cloud.google.com/products/calculator/) para gerar uma estimativa de custo com base no uso projetado.

## Primeiros passos

### Instale a SDK da Vertex AI
**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# ! pip3 install --upgrade --user google-cloud-aiplatform

### **Reinicie o kernel do seu jupyter notebook** 

Como a instalação está sendo realizada com a opção `--user` é necessário reiniciar o kernel para que os novos módulos se tornem acessíveis.

**Importante:** só descomente a linha abaixo se você **não estiver** executando este laboratório no Qwiklabs

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ O kernel do notebook está sendo reiniciado. Por favor aguarde este processo ser finalizado antes de continuar com os próximos passos. ⚠️</b>
</div>

### **Somente para uso no Colab - Autentique o seu ambiente de notebook** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para realizar a autenticação da sua sessão de notebook com a Google Cloud Esse passo é importante **para utilização no Colab** para garantir que as chamadas a APIs de Google Cloud funcionem sem problemas.

In [ ]:
# import sys

# # Additional authentication is required for Google Colab
# if "google.colab" in sys.modules:
#     # Authenticate user to Google Cloud
#     from google.colab import auth

#     auth.authenticate_user()

### **Somente para uso no Colab - defina o projeto Google Cloud a ser utilizado** 

Caso você esteja executando este notebook no Google Colab, descomente a célula abaixo para definir qual projeto Google Cloud será utilizado pelo Colab na execução deste notebook. Senão, siga para as próximas instruções.

In [ ]:
# if "google.colab" in sys.modules:
#     # Define project information
#     PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
#     LOCATION = "us-central1"  # @param {type:"string"}

#     # Initialize Vertex AI
#     import vertexai

#     vertexai.init(project=PROJECT_ID, location=LOCATION)

### Importe as bibliotecas necessárias

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import warnings
warnings.simplefilter("ignore", UserWarning)

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmCategory, 
    HarmBlockThreshold,
    Image,
    Part,
)

## Importe o modelo `Gemini 1.0 Pro`

O Gemini Pro (`gemini-1.0-pro`) ajuda na realização de tarefas utilizando linguagem natural, chats multiturno de texto e código e para a geração de código.

In [2]:
model = GenerativeModel("gemini-1.0-pro")

# Set parameters to reduce variability in responses
generation_config = GenerationConfig(
    temperature=0,
    top_p=0.1,
    top_k=1,
    max_output_tokens=1024,
)

## Gere um resultado e veja a classificação do resultado

Comece gerando uma resposta de texto com som agradável usando o Gemini.

In [3]:
# Call Gemini API
nice_prompt = "Escreva três coisas legais sobre os seres humanos"
response = model.generate_content(
    contents=[nice_prompt],
    generation_config=generation_config,
)

print(response.text, end="")

1. **Capacidade de empatia e compaixão:** Os seres humanos têm uma capacidade única de se colocar no lugar dos outros, entender seus sentimentos e oferecer apoio emocional.

2. **Criatividade e inovação:** Os seres humanos são altamente criativos e inovadores, capazes de gerar novas ideias, resolver problemas e criar obras de arte, tecnologia e cultura.

3. **Resiliência e adaptabilidade:** Os seres humanos são incrivelmente resilientes e adaptáveis, capazes de superar desafios, aprender com experiências e se ajustar a novas situações.

#### Cheque a classificação

Veja as informações de `safety_ratings` da resposta gerada.

In [4]:
response = model.generate_content(
    contents=[nice_prompt],
    generation_config=generation_config,
)

print(response)

candidates {
  content {
    role: "model"
    parts {
      text: "1. **Capacidade de empatia e compaix\303\243o:** Os seres humanos t\303\252m uma capacidade \303\272nica de se colocar no lugar dos outros, entender seus sentimentos e oferecer apoio e conforto.\n\n2. **Criatividade e inova\303\247\303\243o:** Os seres humanos s\303\243o altamente criativos e inovadores, capazes de gerar novas ideias, resolver problemas e criar obras de arte, tecnologia e cultura.\n\n3. **Resili\303\252ncia e adaptabilidade:** Os seres humanos s\303\243o incrivelmente resilientes e adapt\303\241veis, capazes de superar desafios, aprender com experi\303\252ncias e se ajustar a novas circunst\303\242ncias."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGI

#### Compreendendo a classificação de segurança: categoria e probabilidade

Você pode ver as classificações de segurança, incluindo cada tipo de `categoria` e seu rótulo de `probabilidade` associado.

Os tipos de `categoria` incluem:

* Assédio: `HARM_CATEGORY_HARASSMENT`
* Discurso de ódio: `HARM_CATEGORY_HATE_SPEECH`
* Declarações sexualmente explícitas: `HARM_CATEGORY_SEXUALLY_EXPLICIT`
* Conteúdo perigoso: `HARM_CATEGORY_DANGEROUS_CONTENT`

Os rótulos de `probabilidade` são:

* `NEGLIGIBLE` - o conteúdo tem uma probabilidade insignificante de ser inseguro
* `LOW` - o conteúdo tem baixa probabilidade de ser inseguro
* `MEDIUM` - o conteúdo tem uma probabilidade média de ser inseguro
* `HIGH` - o conteúdo tem uma alta probabilidade de ser inseguro

Experimente um prompt que possa acionar uma destas categorias:

In [5]:
impolite_prompt = "Escreva uma lista de cinco coisas disrespeitosas que podem ser ditas quando se bate o dedinho do pé em uma quina de pesa:"

impolite_response = model.generate_content(
    impolite_prompt,
    generation_config=generation_config,
)

print(impolite_response)

candidates {
  content {
    role: "model"
    parts {
      text: "1. \"Isso n\303\243o foi nada, seu beb\303\252 chor\303\243o.\"\n2. \"Voc\303\252 \303\251 t\303\243o desajeitado, n\303\243o consegue nem andar direito.\"\n3. \"Pare de reclamar, \303\251 s\303\263 um dedinho.\"\n4. \"Voc\303\252 est\303\241 exagerando, n\303\243o \303\251 t\303\243o ruim assim.\"\n5. \"Voc\303\252 deveria estar agradecido por n\303\243o ter quebrado o p\303\251 inteiro.\""
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: LOW
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 33
  candidates_token_count: 78
  total_token_count: 111
}



#### Respostas bloqueadas

Se a resposta for bloqueada, você verá que o candidato final inclui `blocked: true` e também observará qual das classificações de segurança desencadeou o bloqueio da resposta (por exemplo, `finish_reason: SAFETY`).

In [6]:
rude_prompt = "Escreva uma lista de coisas rudes que podem ser faladas quando se bate o dedinho do pé em uma quina de mesa:"

rude_responses = model.generate_content(
    rude_prompt,
    generation_config=generation_config,
)

print(rude_responses)

candidates {
  content {
    role: "model"
  }
  finish_reason: SAFETY
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: MEDIUM
    blocked: true
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 29
  total_token_count: 29
}



### Definindo limites para classificações de segurança

Talvez você queira ajustar os limites padrão do filtro de segurança dependendo das políticas de negócios ou do caso de uso. A API Vertex AI Gemini oferece uma maneira de ultrapassar um limite para cada categoria.

A lista abaixo mostra os possíveis rótulos de limite:

* `BLOCK_ONLY_HIGH` - bloqueia quando alta probabilidade de conteúdo inseguro é detectada
* `BLOCK_MEDIUM_AND_ABOVE` - bloqueia quando uma probabilidade média ou alta de conteúdo é detectada
* `BLOCK_LOW_AND_ABOVE` - bloqueia quando probabilidades baixas, médias e altas de conteúdo inseguro são detectadas
* `BLOCK_NONE` - sempre mostra, independentemente da probabilidade de conteúdo inseguro

#### Defina limites de segurança
Abaixo, os limites de segurança foram definidos para o limite mais sensível: `BLOCK_LOW_AND_ABOVE`

In [7]:
safety_settings={
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }

#### Test thresholds

Here you will reuse the impolite prompt from earlier together with the most sensitive safety threshold. It should block the response even with the `LOW` probability label.

In [8]:
rude_prompt = "Escreva uma lista de coisas rudes que podem ser faladas quando se bate o dedinho do pé em uma quina de mesa:"

impolite_response = model.generate_content(
    rude_prompt,
    generation_config=generation_config,
    safety_settings=safety_settings,
)

print(impolite_response)

candidates {
  content {
    role: "model"
  }
  finish_reason: SAFETY
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: MEDIUM
    blocked: true
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
}
usage_metadata {
  prompt_token_count: 29
  total_token_count: 29
}

